# Project 1 - Index Analytics

Bowen Chen

January 18, 2018

## Executive Summary

In this index analytics project, the S&P 500 index's performance is evaluated based on its geometric and arithmetic averages for different frequencies - daily, monthly, annual and 5-year periods. This project also evaluated the performances of 4 different self financing portfolios in the forms of excess returns for the same 4 frequencies as S&P 500 index performances. From the analysis, we can draw the conclusion that arithmetic average returns will always be higher than the geometric average returns, with the 5-year period arithmetic returns the highest among all frequencies (**~18.78%** for S&P500 performances, **~11.08%** for self financing portfolios). However, **the 5-year return also largely depends on the assumptions imposed in the calculations**. The overlapping of the long-time series definitly affects the performance outlook of the investments, providing a great trick for funds to manipulate their performance metrics. 

## Key Questions to Answer

* For the (CRSP) S&P500 with dividends, what was the average arithmetic and geometric historical mean rate of return for daily returns, for monthly returns, for annual returns, and for 5-year returns from 1/1/1973 through 1/1/2015? 


* How would each of the four self-financed strategies with the prevailing short rate, the 30-day Treasury rate, the 1-year Treasury rate, and the 5-year Treasury rates, respectively have performed in terms of excess return?


* Does overlapping the longer-term series lead to different inference?

## Computations

#### Import S&P 500, Risk - free Data from WRDS

In [779]:
library('data.table')
library('dplyr')
library('lubridate')
library('magrittr')
library('zoo')

In [780]:
# Import S&P500 data and convert the date column to date object
sp500_return = fread('data/SP500Return.csv')
sp500_return[,caldt := as.Date(caldt, format = '%m/%d/%Y')]

## Geometric Means and Arithmetic Means

Next, we proceed to answering the first question,

*For the (CRSP) S&P500 with dividends, what was the average arithmetic and geometric historical mean rate of return for daily returns, for monthly returns, for annual returns, and for 5-year returns from 1/1/1973 through 1/1/2015?*

### Daily Returns

#### Annualized daily geometric returns 

From 1/1/1973 through 1/1/2015, there are in total of 10617 days in the sample. The daily geometric means could be found by

$$ r_{geometric} = [(1 + r_1)(1 + r_2)\cdots(1 + r_{n_{days}})] ^{\frac{1}{n_{days}}} - 1$$

$n_{days}$ represents the number of days in the sample

In [781]:
# daily geometric mean
num_days = length(sp500_return[, caldt])
daily_geo_return = (prod(sp500_return[, vwretd] + 1))^(1/num_days) - 1

Annualize daily geometric return, $r_{annualized} = (1 + r_{daily}) ^ {252} - 1$

In [782]:
annualized_daily_geo_return = (1 + daily_geo_return) ^ 252 - 1

#### Annualized daily artithmetic returns

The arithmetic means could be found by

$$ r_{arithmetic} = \frac{r_1 + r_2 + r_3 + \cdots + r_{n_{days}}}{n_{days}}$$

In [783]:
# daily arithmetic mean
num_days = length(sp500_return[, caldt])
daily_arith_return = sum(sp500_return[, vwretd])/num_days

Annualize the arithmetic return, $r_{annualized} =  r_{daily} \times 252$

In [784]:
annualized_daily_arithm_return = daily_arith_return * 252

In [785]:
daily_return = cbind(annualized_daily_geo_return,
                        annualized_daily_arithm_return)

### Monthly Returns

#### Annualized monthly geometric returns 

From 1/1/1973 through 1/1/2015, there are in total of 504 months in the sample. The monthly geometric means could be found by

$$ r_{geometric} = [(1 + r_1)(1 + r_2)\cdots(1 + r_{n_{months}})] ^{\frac{1}{n_{months}}} - 1$$

$n_{months}$ represents the number of months in the sample. We need to first convert the data into monthly frequency. 

In [786]:
# Convert data into monthly frequency
sp500_monthly = sp500_return %>% 
                    group_by(month=floor_date(caldt, "month")) %>% 
                    summarise(vwretd = (function(r) prod(1 + r))(vwretd))%>%
                    as.data.table() 

In [787]:
# monthly geometric mean
num_months = length(sp500_monthly[,month])
monthly_geo_return = (prod(sp500_monthly[, vwretd]))^(1/num_months) - 1

Annualize monthly geometric return, $r_{annualized} = (1 + r_{monthly}) ^ {12} - 1$

In [788]:
annualized_monthly_geo_return = (1 + monthly_geo_return)^12 - 1

#### Annualized monthly arithmetic returns

The monthly arithmetic means could be found by

$$ r_{arithmetic} = \frac{r_1 + r_2 + r_3 + \cdots + r_{n_{motnhs}}}{n_{months}}$$ 

We could use the same monthly frequency data that is used to calculate monthly geometric mean.

In [789]:
# monthly arithmetic mean
num_months = length(sp500_monthly[,month])
monthly_arithm_return = sum(sp500_monthly[, vwretd] - 1)/num_months

Annualize the arithmetic return, $r_{annualized} =  r_{monthly} \times 12$

In [790]:
annualized_monthly_arithm_return = monthly_arithm_return * 12

In [791]:
monthly_return = cbind(annualized_monthly_geo_return,
                        annualized_monthly_arithm_return)

### Annual Returns

#### Annual geometric returns 

From 1/1/1973 through 1/1/2015, there are in total of 42 years. The annual geometric means could be found by

$$ r_{geometric} = [(1 + r_1)(1 + r_2)\cdots(1 + r_{n_{years}})] ^{\frac{1}{n_{years}}} - 1 $$

$n_{years}$ represents number of years in the sample. We need to first convert the data into annual frequency, but annualization is no longer needed. Same as monthly returns, due to the limitation of the *group_by* opreation in R, we will need to subtract 1 from the total months since 1/1/2015 does not count as an extra year

In [792]:
# Convert data into annual frequency
sp500_annual = sp500_return %>% 
                    group_by(year=floor_date(caldt, "year")) %>% 
                    summarise(vwretd = (function(r) prod(1 + r))(vwretd))%>%
                    as.data.table()

In [793]:
# annual geometric mean
num_years = length(sp500_annual[,year])
annual_geo_return = prod(sp500_annual[, vwretd])^(1/num_years) - 1

#### Annualized arithmetic returns

The arithmetic means could be found by

$$ r_{arithmetic} = \frac{r_1 + r_2 + r_3 + \cdots + r_{n_{years}}}{n_{years}}$$ 

We could use the same monthly frequency data that is used to calculate annual geometric mean

In [794]:
# annual arithmetic mean
num_years = length(sp500_annual[,year])
annual_arithm_return = sum(sp500_annual[, vwretd]- 1)/num_years

In [795]:
annual_return = cbind(annual_geo_return,
                        annual_arithm_return)

### 5 Year Returns

There are in total of 42 years in this dataset, which indicates that there will be 2 extra years will not be able to make a full 5-year period. The two extra years will likely affect the magnitude of geometric return, therefore I decided to remove the year of 2014 and 2015 from the dataset, making it 8 total 5-year periods.  The annual geometric means could be found by

$$ r_{geometric} = [(1 + r_1)(1 + r_2)\cdots(1 + r_{n_{5\_years}})] ^{\frac{1}{n_{5\_years}}} - 1$$

$n_{5\_years}$ represents number of years in the sample. We need to first convert the data into 5-year frequency.

In [796]:
# remove the year 2013 and 2014
sp500_return_short = sp500_return[year(caldt) >= 1975]

In [797]:
# Convert data into 5-year frequency
sp500_5_year = sp500_return_short%>% 
                    group_by(yr5 = floor_date(caldt, "5 years")) %>% 
                    summarise(vwretd = (function(r) prod(1 + r))(vwretd))%>%
                    as.data.table()

In [798]:
# 5 yr geometric mean
num_5_years = length(sp500_5_year[,yr5])
five_yr_geo_return = prod(sp500_5_year[, vwretd])^(1/num_5_years) - 1

Annualize 5-year geometric return, $r_{annualized} = (1 + r_{5\_year}) ^ {\frac{1}{5}} - 1$

In [799]:
annualized_5yr_geo_return = (1 + five_yr_geo_return)^(1/5) - 1

The arithmetic means could be found by

$$ r_{arithmetic} = \frac{r_1 + r_2 + r_3 + \cdots + r_{n_{5\_years}}}{n_{5\_years}}$$ 

We could use the same monthly frequency data that is used to calculate 5-year geometric mean

In [800]:
# monthly arithmetic mean
num_5_yr = length(sp500_5_year[,yr5])
five_yr_arithm_return = sum(sp500_5_year[, vwretd]- 1)/num_5_yr

Annualize 5-year geometric return, $r_{annualized} =  r_{5\_year}\times \frac{1}{5}$

In [801]:
annualized_5yr_arithm_return = five_yr_arithm_return/5

In [802]:
yr5_return = cbind(annualized_5yr_geo_return,
                        annualized_5yr_arithm_return)

**Summary:** 

We will build a summary table for different peirod returns for S&P500 to see if there is anything interesting 

In [803]:
sp500_mean_returns = rbind(daily_return * 100, 
                           monthly_return * 100, 
                           annual_return * 100, 
                           yr5_return * 100)%>% as.data.table() %>%
                           setNames(c('Geometric Return (%)', 
                                          'Arithmetic Return (%)'))
                            
sp500_mean_returns[, ` `:=c('Daily', 'Monthly', 'Annual', '5 Year')]
setcolorder(sp500_mean_returns, c(3,1,2))
# Display Summary Table
print(sp500_mean_returns)

           Geometric Return (%) Arithmetic Return (%)
1:   Daily             10.33694              11.33416
2: Monthly             10.35028              11.09121
3:  Annual             10.35028              11.92816
4:  5 Year             12.20949              18.78374


From the summary table above, following conclusions could be drawn

* arithmetic returns are always larger than geometric returns, regardless the period chosen as base of the calculations


* annual geometric returns in 1973 and 1974 are negative, therefore the values 5-year return would be lower if the year 1973 and 1974 is included 


* **geometric return will be 9.77%**, and **arithmetic return will be 13.91%** if the **last two years (2013, 2014) are removed instead of the first two years (1973, 1974)**


* removing the last two years could also be an option, and this assumption will result in a totally different result for both 5-year geometric and arithmetic returns


* 5 year arithmetic return is the largest in value among different periods - there is a steep increase of average returns for 5 year

## Self Financing Portfolios

We will proceed to answer the second question,

*How would each of the four self-financed strategies with the prevailing short rate, the 30-day Treasury rate, the 1-year Treasury rate, and the 5-year Treasury rates, respectively have performed in terms of excess return? *

The self-financing strategies are defined as the zero initial investments that our portfolio holds. We will use the treasury bills to finance the investment in stock markets

**Data Import**

The prevailing short rate, 30-day, 1-year and 5-year T-bill data are downloaded from WRDS's Federal Reserve repository

## Excess Return for Prevailing Short Rates

The prevailing short rate is the federal overnight rates, the data is availiable in WRDS

In [804]:
short_rate = fread('data/fed_fund_rate.csv')
short_rate[, `:=` (date = as.Date(as.character(date), 
                                   format = '%Y%m%d', 
                                   origin = "1960-10-01"),
                  daily_rf = FF_O/(100*252))]

There are missing data in *short_rate* data table, we should first left join the *short_rate* table to *sp500_return* to make them have the same length

In [805]:
excess_return_daily = merge(sp500_return, short_rate[,c('date', 'daily_rf')], 
                            all.x = T, by.x = 'caldt', by.y = 'date')

The missing data in *daily_rf* column could be imputed using last observation carry forward, which can be accomplished by na.locf function from package 'zoo'. By doing so we assumed the overnight rate does not change from the previous day if the data is missing 

In [806]:
excess_return_daily[,daily_rf := na.locf(daily_rf)]

After all the missing values are imputed, the daily excess return could be found. To find the daily excess return for S&P 500 over risk-free rates, we will need to have take the difference of two series.

In [807]:
excess_return_daily[,excess_r := vwretd - daily_rf]

### Geometric Mean for Daily Excess Returns

The equation is the same as the geometric return equations above

In [808]:
num_days = length(excess_return_daily[, caldt])
daily_geo_excess_return = (prod(excess_return_daily[, excess_r] + 1))^(1/num_days) - 1

In [809]:
annualized_daily_geo_excess_return = (1 + daily_geo_excess_return) ^ 252 - 1

### Arthmetic Mean for Daily Excess Returns
The equation is the same as the arithmetic return equations above

In [810]:
# daily arithmetic mean
num_days = length(excess_return_daily[, caldt])
daily_arith_excess_return = sum(excess_return_daily[, excess_r])/num_days

In [811]:
annualized_daily_arithm_excess_return = daily_arith_excess_return * 252

In [812]:
daily_excess = cbind(annualized_daily_geo_excess_return,
                        annualized_daily_arithm_excess_return)

## 30-day T-Bill Excess Return

In [813]:
# Treasury Bills constant maturity
t_bills = fread("data/t_bills.csv")
t_bills = t_bills[,c("date", "TCMNOM_M1", "TCMNOM_Y1", "TCMNOM_Y5")]

In [814]:
# import the daily promised treasury yield and convert the date column to date objects
# convert the annualized yield to monthly frequency
t_bill_30_day = t_bills[,c("date", "TCMNOM_M1")]

t_bill_30_day[, `:=` (month = as.Date(as.character(date), 
                                   format = '%Y%m%d', 
                                   origin = "1960-10-01"),
                   t30ret = TCMNOM_M1/(100*12))]

t_bill_30_day = t_bill_30_day[month >= '2001/07/01', c('month', 't30ret')]

To find the excess return for S&P 500 over risk-free rates, we will need to have take the difference of two series. Since the two series have difference frequencies, we will convert the daily S&P 500 return to monthly S&P 500 return, which is stored in *sp500_monthly* dataset. Unfortunately the 30-day T-bill data is only avaliable after 2001/07/31, the period chosen for this calculations are after 2001/08/01

In [815]:
# subtract 1 to make to obtain return
sp500_monthly[month >= '2001/07/01', vwretd:= vwretd - 1]

We have a little issue here with the different dates,

In the sp500_monthly data table, the monthly data is quoted as the beginning of the month while the risk_free data table quotes the monthly data as the end of the month,

In [816]:
print(t_bill_30_day[1:5,])

        month      t30ret
1: 2001-07-31 0.003058333
2: 2001-08-31 0.002941667
3: 2001-09-30 0.002233333
4: 2001-10-31 0.001891667
5: 2001-11-30 0.001658333


Therefore, we will need to convert the month-end date to the month-beginning date using floor_date

In [817]:
t_bill_30_day[,month := floor_date(month, "month")]
print(t_bill_30_day[1:5,])

        month      t30ret
1: 2001-07-01 0.003058333
2: 2001-08-01 0.002941667
3: 2001-09-01 0.002233333
4: 2001-10-01 0.001891667
5: 2001-11-01 0.001658333


Merge two data tables and impute the missing values

In [818]:
# merge the two data tables
excess_return_monthly = merge(sp500_monthly, t_bill_30_day, 
                                by.x = 'month', by.y = 'month')

# Fill previous month value to missing value
excess_return_monthly[,t30ret := na.locf(t30ret)]

In [819]:
# build excess return column
excess_return_monthly[, excess_r :=  vwretd - t30ret]

### Geometric Mean for 30-day Excess Returns

The equation is the same as the geometric return equations above

In [820]:
num_months = length(excess_return_monthly[,month])
monthly_geo_excess_return = (prod(1+ excess_return_monthly[, excess_r]))^(1/num_months) - 1

In [821]:
annualized_monthly_geo_excess_return = (1 + monthly_geo_excess_return)^12 - 1

### Arthmetic Mean for 30-day Excess Returns
The equation is the same as the arithmetic return equations above

In [822]:
num_months = length(excess_return_monthly[,month]) - 1
monthly_arithm_excess_return = sum(excess_return_monthly[, excess_r])/num_months

In [823]:
annualized_monthly_arithm_excess_return = sum(monthly_arithm_excess_return)*12

In [824]:
monthly_excess = cbind(annualized_monthly_geo_excess_return,
                        annualized_monthly_arithm_excess_return)

## 1-year Excess Return

The 1-year treasury yield data is also availiable on WRDS

In [825]:
t_bill_1yr = t_bills[,c("date", "TCMNOM_Y1")]

t_bill_1yr[, `:=` (month = as.Date(as.character(date), 
                                   format = '%Y%m%d', 
                                   origin = "1960-10-01"),
                   b1ret = TCMNOM_Y1/(100*12))]

t_bill_1yr = t_bill_1yr[, c('month', 'b1ret')]

The 1-year treasury yield is quoted in monthly frequency, therefore should be adjusted to match the 1 year period of *sp500_annual*

In [826]:
# Convert data into annual frequency
t_bill_1yr = t_bill_1yr%>% 
                    group_by(year = floor_date(month, "year")) %>% 
                    summarise(b1ret = (function(r) prod(1 + r))(b1ret))%>%
                    as.data.table()

1-year excess return could be found by merging with *sp500_annual*, missing values will again be imputed

In [827]:
# merge the two data tables
annual_excess_return = merge(sp500_annual, t_bill_1yr, 
                                 by.x = 'year', by.y = 'year')
# impute missing values
annual_excess_return[, b1ret := na.locf(b1ret)]

In [828]:
# build excess return column
annual_excess_return[, excess_r :=  vwretd - b1ret]

### Geometric Mean for 1-year Excess Returns

In [829]:
num_years = length(annual_excess_return[,year])
annual_geo_excess_return = (prod( 1+ annual_excess_return[, excess_r]))^(1/num_years) - 1

### Arthmetic Mean for 1-year Excess Returns

In [830]:
num_years = length(annual_excess_return[,year])
annual_arithm_excess_return = sum(annual_excess_return[, excess_r])/num_years

In [831]:
annual_excess = cbind(annual_geo_excess_return, 
                          annual_arithm_excess_return)

## 5-year Excess Return

Same as the 1-year excess returns, we have to find the corresponding 5 year T-bil rate on WRDS

In [832]:
t_bill_5yr = t_bills[,c("date", "TCMNOM_Y5")]

t_bill_5yr[, `:=` (month = as.Date(as.character(date), 
                                   format = '%Y%m%d', 
                                   origin = "1960-10-01"),
                   b5ret = TCMNOM_Y5/(100*12))]

t_bill_5yr = t_bill_5yr[year(month) >= 1975, c('month', 'b5ret')]

The 5-year treasury yield is quoted in annual frequency, therefore should be adjusted to match the 5 year period of *sp500_5_year*

In [833]:
# Convert data into annual frequency
t_bill_5yr = t_bill_5yr%>% 
                    group_by(yr5 = floor_date(month, "5 years")) %>% 
                    summarise(b5ret = (function(r) prod(1 + r))(b5ret))%>%
                    as.data.table()

5-year excess return could be found by merging with *sp500_5_year*

In [834]:
# merge the two data tables
yr5_excess_return = merge(sp500_5_year, t_bill_5yr, by.x = 'yr5', by.y = 'yr5')

# build excess return column
yr5_excess_return[, excess_r :=  vwretd - b5ret]

### Geometric Mean for 5-year Excess Returns

In [835]:
num_5_years = length(yr5_excess_return[,yr5])
yr5_geo_excess_return = (prod( 1 + yr5_excess_return[, excess_r]))^(1/num_5_years) - 1

In [836]:
annualized_5yr_geo_return = (1 + yr5_geo_excess_return)^(1/5) - 1

### Arthmetic Mean for 1-year Excess Returns

In [837]:
num_5_years = length(yr5_excess_return[,yr5])
yr5_arithm_excess_return = sum(yr5_excess_return[, excess_r])/num_5_years

In [838]:
annualized_5yr_arithm_return = sum(yr5_arithm_excess_return)/5

In [839]:
yr5_excess = cbind(annualized_5yr_geo_return,
                        annualized_5yr_arithm_return)

**Summary:** 

We will build a summary return table for different period excess returns

In [840]:
excess_mean_returns = rbind(daily_excess * 100, 
                           monthly_excess * 100, 
                           annual_excess * 100, 
                           yr5_excess * 100) %>% as.data.table() %>%
                           setNames(c('Geometric Excess Return (%)', 
                                          'Arithmetic Excess Return (%)'))
                            
excess_mean_returns[, ` `:=c('Daily', 'Monthly', 'Annual', '5 Year')]
setcolorder(excess_mean_returns, c(3,1,2))

# Display Summary Table
print(excess_mean_returns)

           Geometric Excess Return (%) Arithmetic Excess Return (%)
1:   Daily                    4.293887                     5.700962
2: Monthly                    4.572426                     5.637794
3:  Annual                    4.428523                     6.119932
4:  5 Year                    6.661580                    11.018330


We will have a similar conclusion this time,

* 5-year arithmetic returns remains the largest among all return values - way larger than the returns calculated from the other 3 frequencies


* again, assumptions imposed are very important here - the missing data in daily returns could be imputed by different ways (or even removed from the series), while the **5-year returns can be totally different** if **different two years are removed (for example 2013 and 2014)**


* with the assumptions right now, the 5-year arithmetic excess returns still blows up due to the large S&P 500 5-year arithmetic returns

# Conclusion

Based on the analysis above, we could draw the conclusion that arithmetic return could be the more attractive value for funds to show to their investors than geometric returns. When there are non-divisible periods of data avaliable, the assumptions imposed would largely affect the result of the calculation. The overlapping time series could greatly affect the performance outlook for a certain investment, but geometric returns are the ones that investors could really retain. Therefore, investors should be careful with the quoted return values when choosing their investments of different funds.Going back to the last key question, 

*Does overlapping the longer-term series lead to different inference? *

The answer is definitely a **yes**.